# Import Packages


In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import datetime
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1)

# Team Records

In [2]:
# Collect first page of artists’ list
page = requests.get('http://stats.espncricinfo.com/ci/engine/records/index.html')
# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'lxml')
RecordByTm= soup.find(id = 'RecordByTm').find(id = 'recteam')
TeamName = RecordByTm.find_all('a', class_ = 'RecordLinks')
# Create for loop to print out all team names
for name in TeamName:
    names = name.contents[0]
    print(names)

 Afghanistan
 Australia
 Bangladesh
 England
 ICC World XI
 India
 Ireland
 New Zealand
 Pakistan
 South Africa
 Sri Lanka
 West Indies
 Zimbabwe
 Africa XI
 Asia XI
 Bermuda
 Canada
 East Africa
 Hong Kong
 Kenya
 Namibia
 Nepal
 Netherlands
 Oman
 Papua New Guinea
 Scotland
 United Arab Emirates
 United States of America


# LIST OF MATCH RESULTS (BY YEAR)
## ONE-DAY INTERNATIONALS


In [3]:
(Team_1, Team_2, Winner, Margin, Ground, Ground_url, Match_Date, Scorecard, Scorecard_url) = ([],[],[],[],[],[],[],[],[])

now =datetime.datetime.now()
max_Year = now.year


for year in range(1971,max_Year+1):
    # Collect first page of list
    url = 'http://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id='+str(year)+';type=year'
    page = requests.get(url)
    # Create a BeautifulSoup object
    soup = BeautifulSoup(page.text, 'lxml')
    Match_Result_Year_Table = soup.find('table', class_ = 'engineTable')
    year_Records = Match_Result_Year_Table.find_all('tbody')
    year_Records =BeautifulSoup(str(year_Records), 'lxml')
    for i in range(len(year_Records.find_all('tr'))):
        # print(i , year)
        record =year_Records.find_all('tr')[i]
        td_records = record.find_all('td')
        Team_1.append((td_records[0].text))
        Team_2.append((td_records[1].text))
        Winner.append((td_records[2].text))
        Margin.append((td_records[3].text))
        Ground.append((td_records[4].string))
        Ground_url.append((str(td_records[4]).split(".")[0].split("/")[-1]))
        Match_Date.append((td_records[5].text))
        Scorecard.append((td_records[6].text))
        Scorecard_url.append((str(td_records[6]).split(".")[0].split("/")[-1]))

In [4]:
import pandas as pd
one_day_internationals = pd.DataFrame()
one_day_internationals['Team_1'] = Team_1
one_day_internationals['Team_2'] = Team_2
one_day_internationals['Winner'] = Winner
one_day_internationals['Margin'] = Margin
one_day_internationals['Ground'] = Ground
one_day_internationals['Ground_url'] = Ground_url
one_day_internationals['Match_Date'] = Match_Date
one_day_internationals['Scorecard'] = Scorecard
one_day_internationals['Scorecard_url'] = Scorecard_url
one_day_internationals.count()
# one_day_internationals.Ground_url.unique()
one_day_internationals.to_csv('one_day_internationals.csv')

# Ground Details

In [7]:
one_day_internationals = pd.read_csv('one_day_internationals.csv')
one_day_internationals.head()

,Team_1,Team_2,Winner,Margin,Ground,Ground_url,Match_Date,Scorecard,Scorecard_url
0,Australia,England,Australia,5 wickets,Melbourne,56441,5-Jan-71,ODI # 1,64148
1,England,Australia,England,6 wickets,Manchester,57160,24-Aug-72,ODI # 2,64944
2,England,Australia,Australia,5 wickets,Lord's,57129,26-Aug-72,ODI # 3,64945
3,England,Australia,England,2 wickets,Birmingham,56788,28-Aug-72,ODI # 4,64946
4,New Zealand,Pakistan,New Zealand,22 runs,Christchurch,58812,11-Feb-73,ODI # 5,64149


In [10]:
ground_list = one_day_internationals.Ground_url.unique()
ground_list.sort()
ground_list[1:4]

array([56319, 56322, 56326], dtype=int64)

In [11]:
record_series = pd.DataFrame()
record_ground = pd.DataFrame()
record_series_spam= pd.DataFrame()
record_ground_spam= pd.DataFrame()
for ground_number in ground_list:
    # Collect first page of list
    url = 'http://stats.espncricinfo.com/ci/engine/ground/'+str(ground_number)+'.html?class=2;template=results;type=aggregate'
    # print(ground_number)
    # url = 'http://stats.espncricinfo.com/ci/engine/ground/56319.html?class=2;template=results;type=aggregate'
    page = requests.get(url) 
    soup = BeautifulSoup(page.text, 'lxml')    
    headlinks = soup.find_all('table')[3]
    # print(headlinks)
    headlinks = headlinks.find_all('tr', class_="headlinks")
    headlinks = BeautifulSoup(str(headlinks), 'lxml')
    # print(headlinks.find_all('th'))
    len_headlinks = len(headlinks.find_all('th'))
    # print(len_headlinks)
    if len_headlinks == 14:
        column_list = ['Ground_overall','playing_span','matches_played','matches_won','matches_lost','matches_tied','matches_with_no_result' ,'win_loss_ratio','runs_scored','wickets_taken','balls_bowled','average_runs_per_wicket','average_runs_per_six_balls','profile', 'ground_url']
        records_tag = soup.find_all('tr', class_ ='data1')
        records_tag = list(records_tag)
        # print(records_tag)
        for tr_tag in range(len(records_tag)):
            # print(tr_tag)
            new_records_tag =BeautifulSoup(str(records_tag[tr_tag]), 'lxml')
            # print(new_records_tag)
            new_td_records_tag = new_records_tag.find_all('td')
            # print(new_td_records_tag)
            record_list_spam = []
            for record_tag in new_td_records_tag:
                record_list_spam.append(record_tag.text.strip())
            # print(record_list_spam)
            record_list_spam.append(ground_number)
            record_ground_spam.append(record_list_spam)
            with open('record_ground_spam.csv', 'a') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerow(record_list_spam)
            csvFile.close()
    else:
        column_list = ['Ground_overall','matches_played','matches_won','matches_lost','matches_tied','matches_with_no_result' ,'win_loss_ratio','runs_scored','wickets_taken','balls_bowled','average_runs_per_wicket','average_runs_per_six_balls','profile', 'ground_url']
        records_tag = soup.find_all('tr', class_ ='data1')
        records_tag = list(records_tag)
        # print(records_tag)
        for tr_tag in range(len(records_tag)):
            # print(tr_tag)
            new_records_tag =BeautifulSoup(str(records_tag[tr_tag]), 'lxml')
            # print(new_records_tag)
            new_td_records_tag = new_records_tag.find_all('td')
            # print(new_td_records_tag)
            record_list = []
            for record_tag in new_td_records_tag:
                record_list.append(record_tag.text.strip())
            # print(record_list)
            record_list.append(ground_number)
            record_ground.append(record_list)
            with open('record_ground.csv', 'a') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerow(record_list)
            csvFile.close()

# Cricket Teams Index

In [12]:
url_index_all_cricket_team = "https://www.espncricinfo.com/story/_/id/18791072/all-cricket-teams-index"
page = requests.get(url_index_all_cricket_team)
soup = BeautifulSoup(page.text, 'lxml')
article_body = soup.find('div', class_ = 'article-body')
index_team_p =article_body.find_all('p')
team_url, team_list = ([],[])
for i in range(1,len(index_team_p)):
    team_url.append(index_team_p[i].a['href'])
    team_list.append(index_team_p[i].a.text)
team_index = pd.DataFrame()
team_index['team_url'] = team_url
team_index['team_list'] = team_list
team_index['team_list'] = [x.lower() for x in team_index['team_list']]
team_index['team_list'] = ['-'.join(x.split(' ')) for x in team_index['team_list']]
team_index['team_url'] = 'https://www.espncricinfo.com'+ team_index['team_url'].astype(str)
team_index['team_url'] =[x if x.endswith('/') else x+str('/') for x in team_index['team_url']]
team_index['team_country_number'] = [(x.split('/')[-3]) for x in team_index['team_url']]
team_index.head(2)
team_index.to_csv('team_index.csv')

# Country player Index

In [13]:
player_url, player_name, Team_player = ([],[],[])
for team_name in team_index['team_list']:
    for team_country_number in team_index[team_index['team_list'] == team_name].team_country_number:
        url_team_player = 'http://www.espncricinfo.com/'+str(team_name)+'/content/player/country.html?country='+str(team_country_number)+''
        # print(url_team_player)
        # url_team_player = "http://www.espncricinfo.com/australia/content/player/country.html?country=2"
        page = requests.get(url_team_player)
        soup = BeautifulSoup(page.text, 'lxml')
        playersTable = soup.find('table', class_ = 'playersTable')
        playersTable_tr = playersTable.find_all('td')
        for i in range(len(playersTable_tr)):
            player_url.append(playersTable_tr[i].a['href'])
            player_name.append(playersTable_tr[i].a.text)
            Team_player.append(team_name)
player_index = pd.DataFrame()
player_index['player_url'] = player_url
player_index['player_name'] = player_name
player_index['Team_player'] = Team_player

In [16]:
player_index.head(2)
player_index.shape
player_index.to_csv("player_index.csv")

# Country player batting details

In [17]:
(player_name , class_name, matches_played, innings_batted, not_outs, runs_scored, highest_inns_score, batting_average, balls_faced, batting_strike_rate, hundreds_scored, fifties_scored, boundary_fours, boundary_sixes, catches_taken, stumpings_made) =([],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[])
for player_url in player_index['player_url']:
    url_player = 'http://www.espncricinfo.com'+str(player_url)+''
    # print(url_player)
    # url_player = "http://www.espncricinfo.com/australia/content/player/325026.html"
    page = requests.get(url_player)
    soup = BeautifulSoup(page.text, 'lxml')
    Batting_and_fielding_averages = soup.find_all('table', class_ = 'engineTable')[0]
    Batting_and_fielding_averages =BeautifulSoup(str(Batting_and_fielding_averages), 'lxml')
    Batting_and_fielding_averages= Batting_and_fielding_averages.find('tbody')
    for i in range(len(Batting_and_fielding_averages.find_all('tr'))):
        # print(i)
        record =Batting_and_fielding_averages.find_all('tr')[i]
        td_records = record.find_all('td')
        #print(td_records)
        player_name.append(url_player.split("/")[-1].split(".")[-2])
        class_name.append((td_records[0].text))
        matches_played.append((td_records[1].text))
        innings_batted.append((td_records[2].text))
        not_outs.append((td_records[3].text))
        runs_scored.append((td_records[4].text))
        highest_inns_score.append((td_records[5].text))
        batting_average.append((td_records[6].text))
        balls_faced.append((td_records[7].text))
        batting_strike_rate.append((td_records[8].text))
        hundreds_scored.append((td_records[9].text))
        fifties_scored.append((td_records[10].text))
        boundary_fours.append((td_records[11].text))
        boundary_sixes.append((td_records[12].text))
        catches_taken.append((td_records[13].text))
        stumpings_made.append((td_records[14].text))

In [18]:
player_batting_details = pd.DataFrame()
player_batting_details['player_name'] = player_name
player_batting_details['class_name'] = class_name
player_batting_details['matches_played'] = matches_played
player_batting_details['innings_batted'] = innings_batted
player_batting_details['not_outs'] = not_outs
player_batting_details['runs_scored'] = runs_scored
player_batting_details['highest_inns_score'] = highest_inns_score
player_batting_details['batting_average'] = batting_average
player_batting_details['balls_faced'] = balls_faced
player_batting_details['batting_strike_rate'] = batting_strike_rate
player_batting_details['hundreds_scored'] = hundreds_scored
player_batting_details['fifties_scored'] = fifties_scored
player_batting_details['boundary_fours'] = boundary_fours
player_batting_details['boundary_sixes'] = boundary_sixes
player_batting_details['catches_taken'] = catches_taken
player_batting_details['stumpings_made'] = stumpings_made
player_batting_details.head()

,player_name,class_name,matches_played,innings_batted,not_outs,runs_scored,highest_inns_score,batting_average,balls_faced,batting_strike_rate,hundreds_scored,fifties_scored,boundary_fours,boundary_sixes,catches_taken,stumpings_made
0,272477,ODIs,9,3,2,7,6*,7.00,5,140.00,0,0,0,0,3,0
1,272477,T20Is,7,-,-,-,-,-,-,-,-,-,-,-,4,0
2,272477,First-class,31,45,13,389,39*,12.15,1084,35.88,0,0,39,6,11,0
3,272477,List A,46,24,10,137,24*,9.78,252,54.36,0,0,9,1,11,0
4,272477,T20s,61,12,8,75,26,18.75,69,108.69,0,0,6,1,20,0


In [21]:
print(player_batting_details.shape)
player_batting_details.head()
player_batting_details.to_csv('player_batting_details.csv')

(3162, 16)


# Country player bowling details

In [24]:
player_index = pd.read_csv("player_index.csv")
player_index.head()

,player_url,player_name,Team_player
0,/australia/content/player/272477.html,JP Behrendorff,australia
1,/australia/content/player/326632.html,JA Burns,australia
2,/australia/content/player/326434.html,AT Carey,australia
3,/australia/content/player/261354.html,NM Coulter-Nile,australia
4,/australia/content/player/489889.html,PJ Cummins,australia


In [26]:
(player_name , class_name, matches_played, innings_bowled_in, balls_bowled, runs_conceded, wickets_taken, best_innings_bowling, best_match_bowling, bowling_average, economy_rate, bowling_strike_rate, four_wkts_in_an_inns, five_wkts_in_an_inns, ten_wkts_in_a_match)=([],[],[],[],[],[],[],[],[],[],[],[],[],[],[])

for player_url in player_index['player_url']:
    url_player = 'http://www.espncricinfo.com'+str(player_url)+''
    # url_player = "http://www.espncricinfo.com/australia/content/player/325026.html"
    page = requests.get(url_player)
    soup = BeautifulSoup(page.text, 'lxml')
    Batting_and_fielding_averages = soup.find_all('table', class_ = 'engineTable')[1]
    Batting_and_fielding_averages =BeautifulSoup(str(Batting_and_fielding_averages), 'lxml')
    Batting_and_fielding_averages= Batting_and_fielding_averages.find('tbody')
    print(player_url)
    for i in range(len(Batting_and_fielding_averages.find_all('tr'))):
        # print(i)
        record =Batting_and_fielding_averages.find_all('tr')[i]
        td_records = record.find_all('td')
        #print(td_records)
        player_name.append(url_player.split("/")[-1].split(".")[-2])
        class_name.append((td_records[0].text))
        matches_played.append((td_records[1].text))
        innings_bowled_in.append((td_records[2].text))
        balls_bowled.append((td_records[3].text))
        runs_conceded.append((td_records[4].text))
        wickets_taken.append((td_records[5].text))
        best_innings_bowling.append((td_records[6].text))
        best_match_bowling.append((td_records[7].text))
        bowling_average.append((td_records[8].text))
        economy_rate.append((td_records[9].text))
        bowling_strike_rate.append((td_records[10].text))
        four_wkts_in_an_inns.append((td_records[11].text))
        five_wkts_in_an_inns.append((td_records[12].text))
        ten_wkts_in_a_match.append((td_records[13].text))


/australia/content/player/272477.html
/australia/content/player/326632.html
/australia/content/player/326434.html
/australia/content/player/261354.html
/australia/content/player/489889.html
/australia/content/player/5334.html
/australia/content/player/334337.html
/australia/content/player/434813.html
/australia/content/player/288284.html
/australia/content/player/530011.html
/australia/content/player/215155.html
/australia/content/player/787987.html
/australia/content/player/326637.html
/australia/content/player/272279.html
/australia/content/player/272450.html
/australia/content/player/6683.html
/australia/content/player/325026.html
/australia/content/player/7252.html
/australia/content/player/505110.html
/australia/content/player/774223.html
/australia/content/player/272262.html
/australia/content/player/308798.html
/australia/content/player/7898.html
/australia/content/player/267192.html
/australia/content/player/533042.html
/australia/content/player/311592.html
/australia/content/p

/sri-lanka/content/player/784369.html
/sri-lanka/content/player/623695.html
/sri-lanka/content/player/227772.html
/sri-lanka/content/player/49619.html
/sri-lanka/content/player/49758.html
/sri-lanka/content/player/49764.html
/sri-lanka/content/player/629074.html
/sri-lanka/content/player/49700.html
/sri-lanka/content/player/784373.html
/sri-lanka/content/player/300629.html
/sri-lanka/content/player/49920.html
/sri-lanka/content/player/300631.html
/sri-lanka/content/player/233514.html
/sri-lanka/content/player/653033.html
/sri-lanka/content/player/499594.html
/sri-lanka/content/player/429738.html
/sri-lanka/content/player/237310.html
/sri-lanka/content/player/222354.html
/sri-lanka/content/player/50747.html
/sri-lanka/content/player/301236.html
/sri-lanka/content/player/328026.html
/sri-lanka/content/player/370040.html
/west-indies/content/player/670013.html
/west-indies/content/player/348054.html
/west-indies/content/player/341593.html
/west-indies/content/player/457249.html
/west-indi

/scotland/content/player/297488.html
/scotland/content/player/232435.html
/scotland/content/player/470777.html
/scotland/content/player/461281.html
/scotland/content/player/671805.html
/scotland/content/player/1148466.html
/scotland/content/player/414970.html
/scotland/content/player/458897.html
/scotland/content/player/671823.html
/scotland/content/player/526492.html
/scotland/content/player/353431.html
/scotland/content/player/414971.html
/scotland/content/player/509107.html
/scotland/content/player/659081.html
/scotland/content/player/807535.html
/scotland/content/player/627732.html
/papua-new-guinea/content/player/332980.html
/papua-new-guinea/content/player/25053.html
/papua-new-guinea/content/player/1115033.html
/papua-new-guinea/content/player/524249.html
/papua-new-guinea/content/player/420427.html
/papua-new-guinea/content/player/25051.html
/papua-new-guinea/content/player/646173.html
/papua-new-guinea/content/player/302850.html
/papua-new-guinea/content/player/646175.html
/pa

In [27]:
player_bowling_details = pd.DataFrame()
player_bowling_details['player_name'] = player_name
player_bowling_details['class_name'] = class_name
player_bowling_details['matches_played'] = matches_played
player_bowling_details['innings_bowled_in'] = innings_bowled_in
player_bowling_details['balls_bowled'] = balls_bowled
player_bowling_details['runs_conceded'] = runs_conceded
player_bowling_details['wickets_taken'] = wickets_taken
player_bowling_details['best_innings_bowling'] = best_innings_bowling
player_bowling_details['best_match_bowling'] = best_match_bowling
player_bowling_details['bowling_average'] = bowling_average
player_bowling_details['economy_rate'] = economy_rate
player_bowling_details['bowling_strike_rate'] = bowling_strike_rate
player_bowling_details['four_wkts_in_an_inns'] = four_wkts_in_an_inns
player_bowling_details['five_wkts_in_an_inns'] = five_wkts_in_an_inns
player_bowling_details['ten_wkts_in_a_match'] = ten_wkts_in_a_match
player_bowling_details.shape

(3162, 15)

In [29]:
print(player_bowling_details.shape)
player_bowling_details.to_csv("player_bowling_details.csv")

(3162, 15)


## Ground Details


In [30]:
one_day_internationals.shape

(4183, 9)

In [31]:
one_day_internationals= one_day_internationals[(one_day_internationals['Winner'] != 'no result') & (one_day_internationals['Winner'] != '-')]
one_day_internationals.shape

(4037, 9)

In [52]:
(team_1_name , team_2_name, team_1_score, team_2_score, win_commnet, team_1_fall_of_wickets, 
team_2_fall_of_wickets, series_name, Toss_details, Player_Of_The_Match, Match_number, Season, failed_scoreboard) = ([],[],[],[],[],[],[],[],[],[],[],[],[])
for scorecard in one_day_internationals['Scorecard_url'] :
    url_scoreboard =  'http://stats.espncricinfo.com/ci/engine/match/'+str(scorecard)+'.html'
    print(url_scoreboard)
    #url_scoreboard = "https://www.espncricinfo.com/series/17244/scorecard/64944/england-vs-australia-1st-odi-australia-tour-of-england-and-scotland-1972"
    page = requests.get(url_scoreboard)
    soup = BeautifulSoup(page.text, 'lxml')
    cscore_team = soup.find_all('div', class_ = 'cscore_details')
    cscore_team = BeautifulSoup(str(cscore_team), 'lxml')
    cscore_team = cscore_team.find_all('div', class_ = 'cscore_team icon-font-after')
    try:
        team_1_name.append(cscore_team[0].find('span', class_ = 'cscore_name cscore_name--long').text)
        team_2_name.append(cscore_team[1].find('span', class_ = 'cscore_name cscore_name--long').text)
        team_1_score.append(cscore_team[0].find('div', class_ = 'cscore_score ').text)
        team_2_score.append(cscore_team[1].find('div', class_ = 'cscore_score ').text)
        win_commnet.append(soup.find_all('div', class_ = 'cscore_commentary cscore_commentary--footer')[0].text)
        info =soup.find_all('article', class_ = 'sub-module scorecard')[0]
        info = BeautifulSoup(str(info), 'lxml')
        team_1_fall_of_wickets.append(info.find_all('div', class_="flex-row")[-1].text)
        info =soup.find_all('article', class_ = 'sub-module scorecard')[1]
        info = BeautifulSoup(str(info), 'lxml')
        team_2_fall_of_wickets.append(info.find_all('div', class_="flex-row")[-1].text)
        info =soup.find_all('article', class_ = 'sub-module game-information pre')
        info = BeautifulSoup(str(info), 'lxml')
        series_name.append(info.find_all('div', class_="match-detail--right")[0].text)
        Toss_details.append(info.find_all('div', class_="match-detail--right")[1].text)
        Player_Of_The_Match.append(info.find_all('div', class_="match-detail--right")[2].text)
        Match_number.append(info.find_all('div', class_="match-detail--right")[4].text)
        Season.append(info.find_all('div', class_="match-detail--right")[5].text)
    except (ConnectionError, Exception) as  e:
        failed_scoreboard.append(scorecard)


http://stats.espncricinfo.com/ci/engine/match/667649.html
http://stats.espncricinfo.com/ci/engine/match/1007649.html
http://stats.espncricinfo.com/ci/engine/match/1007651.html
http://stats.espncricinfo.com/ci/engine/match/1007653.html
http://stats.espncricinfo.com/ci/engine/match/593983.html
http://stats.espncricinfo.com/ci/engine/match/593984.html
http://stats.espncricinfo.com/ci/engine/match/593985.html
http://stats.espncricinfo.com/ci/engine/match/1019973.html
http://stats.espncricinfo.com/ci/engine/match/1019975.html
http://stats.espncricinfo.com/ci/engine/match/770121.html
http://stats.espncricinfo.com/ci/engine/match/1019977.html
http://stats.espncricinfo.com/ci/engine/match/770123.html
http://stats.espncricinfo.com/ci/engine/match/667723.html
http://stats.espncricinfo.com/ci/engine/match/667725.html
http://stats.espncricinfo.com/ci/engine/match/770127.html
http://stats.espncricinfo.com/ci/engine/match/667727.html
http://stats.espncricinfo.com/ci/engine/match/667729.html
http://s

http://stats.espncricinfo.com/ci/engine/match/1168115.html
http://stats.espncricinfo.com/ci/engine/match/1168117.html
http://stats.espncricinfo.com/ci/engine/match/1168118.html
http://stats.espncricinfo.com/ci/engine/match/1168119.html
http://stats.espncricinfo.com/ci/engine/match/1004285.html
http://stats.espncricinfo.com/ci/engine/match/635653.html
http://stats.espncricinfo.com/ci/engine/match/635654.html
http://stats.espncricinfo.com/ci/engine/match/635655.html
http://stats.espncricinfo.com/ci/engine/match/635656.html
http://stats.espncricinfo.com/ci/engine/match/635657.html
http://stats.espncricinfo.com/ci/engine/match/1131274.html
http://stats.espncricinfo.com/ci/engine/match/1131275.html
http://stats.espncricinfo.com/ci/engine/match/1131276.html
http://stats.espncricinfo.com/ci/engine/match/1131277.html
http://stats.espncricinfo.com/ci/engine/match/1131278.html
http://stats.espncricinfo.com/ci/engine/match/1131279.html
http://stats.espncricinfo.com/ci/engine/match/914207.html
htt

http://stats.espncricinfo.com/ci/engine/match/730289.html
http://stats.espncricinfo.com/ci/engine/match/730291.html
http://stats.espncricinfo.com/ci/engine/match/885959.html
http://stats.espncricinfo.com/ci/engine/match/1153840.html
http://stats.espncricinfo.com/ci/engine/match/885965.html
http://stats.espncricinfo.com/ci/engine/match/885967.html
http://stats.espncricinfo.com/ci/engine/match/1153841.html
http://stats.espncricinfo.com/ci/engine/match/742619.html
http://stats.espncricinfo.com/ci/engine/match/742621.html
http://stats.espncricinfo.com/ci/engine/match/742623.html
http://stats.espncricinfo.com/ci/engine/match/742625.html
http://stats.espncricinfo.com/ci/engine/match/742627.html
http://stats.espncricinfo.com/ci/engine/match/636159.html
http://stats.espncricinfo.com/ci/engine/match/636160.html
http://stats.espncricinfo.com/ci/engine/match/636161.html
http://stats.espncricinfo.com/ci/engine/match/636162.html
http://stats.espncricinfo.com/ci/engine/match/636163.html
http://stats

http://stats.espncricinfo.com/ci/engine/match/1029821.html
http://stats.espncricinfo.com/ci/engine/match/1144504.html
http://stats.espncricinfo.com/ci/engine/match/1029823.html
http://stats.espncricinfo.com/ci/engine/match/1144505.html
http://stats.espncricinfo.com/ci/engine/match/534209.html
http://stats.espncricinfo.com/ci/engine/match/534210.html
http://stats.espncricinfo.com/ci/engine/match/1144506.html
http://stats.espncricinfo.com/ci/engine/match/1144507.html
http://stats.espncricinfo.com/ci/engine/match/1144508.html
http://stats.espncricinfo.com/ci/engine/match/1144509.html
http://stats.espncricinfo.com/ci/engine/match/534215.html
http://stats.espncricinfo.com/ci/engine/match/534216.html
http://stats.espncricinfo.com/ci/engine/match/911049.html
http://stats.espncricinfo.com/ci/engine/match/534218.html
http://stats.espncricinfo.com/ci/engine/match/534219.html
http://stats.espncricinfo.com/ci/engine/match/911051.html
http://stats.espncricinfo.com/ci/engine/match/1144516.html
http:

http://stats.espncricinfo.com/ci/engine/match/1153248.html
http://stats.espncricinfo.com/ci/engine/match/936163.html
http://stats.espncricinfo.com/ci/engine/match/1153249.html
http://stats.espncricinfo.com/ci/engine/match/657637.html
http://stats.espncricinfo.com/ci/engine/match/936165.html
http://stats.espncricinfo.com/ci/engine/match/657639.html
http://stats.espncricinfo.com/ci/engine/match/936167.html
http://stats.espncricinfo.com/ci/engine/match/657641.html
http://stats.espncricinfo.com/ci/engine/match/1153250.html
http://stats.espncricinfo.com/ci/engine/match/657643.html
http://stats.espncricinfo.com/ci/engine/match/1153251.html
http://stats.espncricinfo.com/ci/engine/match/657645.html
http://stats.espncricinfo.com/ci/engine/match/1153253.html
http://stats.espncricinfo.com/ci/engine/match/1153254.html
http://stats.espncricinfo.com/ci/engine/match/1153255.html
http://stats.espncricinfo.com/ci/engine/match/1157378.html
http://stats.espncricinfo.com/ci/engine/match/1157379.html
http:

http://stats.espncricinfo.com/ci/engine/match/1026849.html
http://stats.espncricinfo.com/ci/engine/match/1026851.html
http://stats.espncricinfo.com/ci/engine/match/518956.html
http://stats.espncricinfo.com/ci/engine/match/518957.html
http://stats.espncricinfo.com/ci/engine/match/518958.html
http://stats.espncricinfo.com/ci/engine/match/518959.html
http://stats.espncricinfo.com/ci/engine/match/518960.html
http://stats.espncricinfo.com/ci/engine/match/518961.html
http://stats.espncricinfo.com/ci/engine/match/518962.html
http://stats.espncricinfo.com/ci/engine/match/518963.html
http://stats.espncricinfo.com/ci/engine/match/518964.html
http://stats.espncricinfo.com/ci/engine/match/518965.html
http://stats.espncricinfo.com/ci/engine/match/518966.html
http://stats.espncricinfo.com/ci/engine/match/518967.html
http://stats.espncricinfo.com/ci/engine/match/518968.html
http://stats.espncricinfo.com/ci/engine/match/518969.html
http://stats.espncricinfo.com/ci/engine/match/518970.html
http://stats

http://stats.espncricinfo.com/ci/engine/match/1144510.html
http://stats.espncricinfo.com/ci/engine/match/474480.html
http://stats.espncricinfo.com/ci/engine/match/474481.html
http://stats.espncricinfo.com/ci/engine/match/1144511.html
http://stats.espncricinfo.com/ci/engine/match/1144512.html
http://stats.espncricinfo.com/ci/engine/match/1144513.html
http://stats.espncricinfo.com/ci/engine/match/1144514.html
http://stats.espncricinfo.com/ci/engine/match/1144515.html
http://stats.espncricinfo.com/ci/engine/match/1144518.html
http://stats.espncricinfo.com/ci/engine/match/1144519.html
http://stats.espncricinfo.com/ci/engine/match/1144520.html
http://stats.espncricinfo.com/ci/engine/match/1144521.html
http://stats.espncricinfo.com/ci/engine/match/1144522.html
http://stats.espncricinfo.com/ci/engine/match/1134033.html
http://stats.espncricinfo.com/ci/engine/match/1134034.html
http://stats.espncricinfo.com/ci/engine/match/1134035.html
http://stats.espncricinfo.com/ci/engine/match/1134036.html

In [53]:
Ground_Details = pd.DataFrame()
Ground_Details['team_1_name'] = team_1_name
Ground_Details['team_2_name'] = team_2_name
Ground_Details['team_1_score'] = team_1_score
Ground_Details['team_2_score'] = team_2_score
Ground_Details['win_commnet'] = win_commnet
Ground_Details['team_1_fall_of_wickets'] = team_1_fall_of_wickets
Ground_Details['team_2_fall_of_wickets'] = team_2_fall_of_wickets
Ground_Details['series_name'] = series_name
Ground_Details['Toss_details'] = Toss_details
Ground_Details['Player_Of_The_Match'] = Player_Of_The_Match
Ground_Details['Match_number'] = Match_number
Ground_Details['Season'] = Season
Ground_Details.shape
Ground_Details = pd.DataFrame(Ground_Details)
Ground_Details.to_csv('Ground_total_score_Details.csv')
Ground_Details.head()


,team_1_name,team_2_name,team_1_score,team_2_score,win_commnet,team_1_fall_of_wickets,team_2_fall_of_wickets,series_name,Toss_details,Player_Of_The_Match,Match_number,Season
0,New Zealand,India,303/5,216 (49.4/50 ov),New Zealand won by 87 runs,"Fall of wickets: 1-22 (Jesse Ryder, 7.4 ov), 2-41 (Martin Guptill, 12.2 ov), 3-193 (Kane Williamson, 37.3 ov), 4-243 (Brendon McCullum, 43.5 ov), 5-274 (Ross Taylor, 47.6 ov)","Fall of wickets: 1-8 (Rohit Sharma, 4.3 ov), 2-20 (Shikhar Dhawan, 9.5 ov), 3-30 (Ajinkya Rahane, 13.2 ov), 4-78 (Ambati Rayudu, 23.6 ov), 5-145 (Virat Kohli, 36.1 ov), 6-167 (Ravichandran Ashwin, 40.2 ov), 7-174 (Ravindra Jadeja, 41.5 ov), 8-181 (MS Dhoni, 44.1 ov), 9-215 (Bhuvneshwar Kumar, 48.6 ov), 10-216 (Varun Aaron, 49.4 ov)",India tour of New Zealand 2013/14,"India , elected to field first",Ross Taylor,ODI no. 3467,2013/14
1,Zimbabwe,India,168,173/1 (42.3/50 ov),India won by 9 wickets (with 45 balls remaining),"Fall of wickets: 1-8 (Peter Moor, 1.6 ov), 2-30 (Hamilton Masakadza, 8.4 ov), 3-47 (Chamu Chibhabha, 13.1 ov), 4-69 (Vusi Sibanda, 19.2 ov), 5-77 (Craig Ervine, 23.6 ov), 6-115 (Sikandar Raza, 37.3 ov), 7-140 (Richmond Mutumbami, 43.3 ov), 8-156 (Graeme Cremer, 46.5 ov), 9-167 (Elton Chigumbura, 49.3 ov), 10-168 (Tendai Chatara, 49.5 ov)","Fall of wickets: 1-11 (Karun Nair, 4.3 ov)",India tour of Zimbabwe 2016,"India , elected to field first",KL Rahul,ODI no. 3742,2016
2,Zimbabwe,India,126,129/2 (26.5/50 ov),India won by 8 wickets (with 139 balls remaining),"Fall of wickets: 1-19 (Hamilton Masakadza, 4.3 ov), 2-21 (Peter Moor, 6.2 ov), 3-39 (Chamu Chibhabha, 9.6 ov), 4-106 (Sikandar Raza, 25.2 ov), 5-106 (Elton Chigumbura, 25.3 ov), 6-107 (Vusi Sibanda, 27.4 ov), 7-112 (Richmond Mutumbami, 30.2 ov), 8-115 (Tendai Chatara, 31.4 ov), 9-126 (Taurai Muzarabani, 34.3 ov)","Fall of wickets: 1-58 (KL Rahul, 14.4 ov), 2-125 (Karun Nair, 26.4 ov)",India tour of Zimbabwe 2016,"India , elected to field first",Yuzvendra Chahal,ODI no. 3744,2016
3,Zimbabwe,India,123,126/0 (21.5/50 ov),India won by 10 wickets (with 169 balls remaining),"Fall of wickets: 1-19 (Hamilton Masakadza, 5.3 ov), 2-55 (Chamu Chibhabha, 20.1 ov), 3-89 (Vusi Sibanda, 28.4 ov), 4-104 (Timycen Maruma, 32.5 ov), 5-104 (Elton Chigumbura, 32.6 ov), 6-104 (Malcolm Waller, 33.1 ov), 7-104 (Graeme Cremer, 33.2 ov), 8-108 (Richmond Mutumbami, 34.3 ov), 9-110 (Tawanda Mupariwa, 36.4 ov), 10-123 (Donald Tiripano, 42.2 ov)","Did not bat: MK Pandey, AT Rayudu, KM Jadhav, MS Dhoni (c) †, AR Patel, DS Kulkarni, JJ Bumrah, BB Sran, YS Chahal",India tour of Zimbabwe 2016,"Zimbabwe , elected to bat first",KL Rahul,India won the 3-match series 3-0,ODI no. 3746
4,West Indies,Zimbabwe,337/4,181/9 (50 ov),West Indies won by 156 runs,"Fall of wickets: 1-168 (Kieran Powell, 28.4 ov), 2-248 (Johnson Charles, 38.6 ov), 3-266 (Andre Russell, 41.3 ov), 4-302 (Dwayne Bravo, 47.2 ov)","Fall of wickets: 1-18 (Vusi Sibanda, 4.4 ov), 2-22 (Chamu Chibhabha, 5.3 ov), 3-31 (Hamilton Masakadza, 9.2 ov), 4-34 (Brendan Taylor, 10.5 ov), 5-92 (Craig Ervine, 23.5 ov), 6-115 (Regis Chakabva, 29.3 ov), 7-156 (Malcolm Waller, 40.6 ov), 8-161 (Kyle Jarvis, 43.4 ov), 9-181 (Natsai M'shangwe, 48.5 ov)",Zimbabwe tour of West Indies 2012/13,"Zimbabwe , elected to field first",Johnson Charles,ODI no. 3337,2012/13
